<a href="https://www.kaggle.com/code/angelchaudhary/revenue-leakage-in-ott-subscription-platforms?scriptVersionId=292967412" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# # Revenue Leakage Detection in OTT Subscription Platforms
## Identifying Failed Payments, Churn Risk, and Hidden Revenue Loss Using Data Science

# Introduction

Subscription-based businesses such as SaaS platforms and OTT services rely heavily on recurring revenue. While customer acquisition often gets the most attention, a significant amount of revenue is silently lost due to issues like failed payments, involuntary churn, and disengaged users who are likely to cancel their subscriptions. This case study focuses on identifying **revenue leakage**—money the company should have earned but didn’t—by analyzing user behavior and payment data. Unlike explicit churn, revenue leakage often goes unnoticed because users may still appear active in the system despite not contributing to revenue.

The business wants to answer three critical questions:
- Where are payments failing, and how frequently does this happen?
- Which users are at high risk of churning in the near future?
- How much revenue is being lost due to failed payments and churn?

Revenue leakage directly impacts profitability but is often overlooked in favor of growth metrics. By proactively identifying churn-risk users and payment failures, companies can:
- Recover lost revenue through payment retries and reminders
- Improve customer retention strategies
- Make data-driven decisions for lifecycle-based interventions

## Approach
In this case study, we will:
1. Perform **Exploratory Data Analysis (EDA)** to understand payment failures and user behavior patterns  
2. Analyze the **customer lifecycle** to identify signals of disengagement  
3. Build a **classification model** to detect users at high risk of churn  
4. Quantify the **revenue impact** of failed payments and churned users  

# LET'S DO IT!!!
![FUNNY GIF](https://media.giphy.com/media/v1.Y2lkPWVjZjA1ZTQ3amM5c2U4emd4cDhsanlpbTJrZnVpNGFyNHludjM0d3poaDd2Mjc1aSZlcD12MV9naWZzX3NlYXJjaCZjdD1n/LWJ7cKyiWPCnVyuAhT/giphy.gif)

## Dataset Overview 
We use a **synthetic Netflix user dataset** containing demographic details, subscription plans, watch behavior, and login activity.  
Since real payment data is not publicly available, payment-related variables are **synthetically engineered** to simulate real-world OTT subscription behavior.

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("smayanj/netflix-users-database")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/netflix-users-database


In [2]:
import pandas as pd
import numpy as np

df = pd.read_csv("/kaggle/input/netflix-users-database/netflix_users.csv")
df.head(2)

,User_ID,Name,Age,Country,Subscription_Type,Watch_Time_Hours,Favorite_Genre,Last_Login
0,1,James Martinez,18,France,Premium,80.26,Drama,2024-05-12
1,2,John Miller,23,USA,Premium,321.75,Sci-Fi,2025-02-05


In [3]:
df.describe()

,User_ID,Age,Watch_Time_Hours
count,25000.000000,25000.000000,25000.000000
mean,12500.500000,46.482880,500.468858
std,7217.022701,19.594861,286.381815
min,1.000000,13.000000,0.120000
25%,6250.750000,29.000000,256.567500
50%,12500.500000,46.000000,501.505000
75%,18750.250000,63.000000,745.732500
max,25000.000000,80.000000,999.990000


## Feature Engineering

In [4]:
# Convert Last_Login to datetime
df["Last_Login"] = pd.to_datetime(df["Last_Login"])

# Assume today's date for analysis
analysis_date = df["Last_Login"].max()

# Calculate days since last login
df["days_since_last_login"] = (analysis_date - df["Last_Login"]).dt.days

df[["User_ID", "days_since_last_login"]].head()

,User_ID,days_since_last_login
0,1,300
1,2,31
2,3,43
3,4,348
4,5,53


Users show a wide range of inactivity, from recently active users to those inactive for several months. High values of days since last login indicate potential disengagement and increased churn risk.

## Map Subscription Plans to Monthly Revenue

In [5]:
# Map subscription plans to monthly fees
plan_price_map = {"Basic": 199,"Standard": 499,"Premium": 649}
df["monthly_fee"] = df["Subscription_Type"].map(plan_price_map)
df[["Subscription_Type", "monthly_fee"]].drop_duplicates()

,Subscription_Type,monthly_fee
0,Premium,649
2,Basic,199
4,Standard,499


Subscription plans are clearly mapped to distinct monthly fees, enabling direct estimation of expected revenue per user. This forms the basis for quantifying revenue loss at an individual and aggregate level.

## Simulate Payment Failure (Core to Revenue Leakage)

In [6]:
# Simulate payment failure probability
np.random.seed(42)

df["payment_failed"] = np.where((df["days_since_last_login"] > 30) & (df["Watch_Time_Hours"] < 100),np.random.choice([0, 1], size=len(df), p=[0.4, 0.6]),np.random.choice([0, 1], size=len(df), p=[0.85, 0.15]))
df["payment_failed"].value_counts(normalize=True)

payment_failed
0    0.81408
1    0.18592
Name: proportion, dtype: float64

Approximately 18–19% of users experience simulated payment failures, indicating a meaningful portion of revenue at risk. This aligns with real-world OTT scenarios where inactive users are more likely to fail renewal payments.

## Calculate Revenue Loss

In [7]:
# Expected vs received revenue
df["revenue_expected"] = df["monthly_fee"]
df["revenue_received"] = np.where(df["payment_failed"] == 1, 0, df["monthly_fee"])
df["revenue_loss"] = df["revenue_expected"] - df["revenue_received"]

df[["revenue_expected", "revenue_received", "revenue_loss"]].head()

,revenue_expected,revenue_received,revenue_loss
0,649,649,0
1,649,649,0
2,199,0,199
3,649,0,649
4,499,499,0


For users with successful payments, expected and received revenue match exactly. For users with failed payments, the entire monthly subscription fee is lost, directly contributing to revenue leakage. Revenue loss is binary at the user level, equal to the full subscription fee when a payment fails.
This allows straightforward aggregation of total leakage and identification of high-impact user segments.

## Total Revenue Leakage

In [8]:
total_expected_revenue = df["revenue_expected"].sum()
total_received_revenue = df["revenue_received"].sum()
total_revenue_loss = df["revenue_loss"].sum()

leakage_percentage = (total_revenue_loss / total_expected_revenue) * 100

total_expected_revenue, total_received_revenue, total_revenue_loss, leakage_percentage

(np.int64(11228500),
 np.int64(9162798),
 np.int64(2065702),
 np.float64(18.396954179097833))

The platform has an expected monthly revenue of approximately 11.23 million, out of which about 2.07 million is lost due to failed payments. This results in a revenue leakage of roughly 18.4%, indicating a significant recurring revenue risk for the platform.

## Revenue Leakage by Subscription Plan

In [9]:
leakage_by_plan = (
    df.groupby("Subscription_Type")[["revenue_expected", "revenue_loss"]]
    .sum()
    .reset_index()
)

leakage_by_plan

,Subscription_Type,revenue_expected,revenue_loss
0,Basic,1662844,316211
1,Premium,5452898,965063
2,Standard,4112758,784428


Premium subscriptions contribute the highest absolute revenue loss, followed by Standard and Basic plans. Although Basic plans have lower individual prices, Premium and Standard plans drive higher leakage due to their higher subscription fees and larger revenue base.

## Who Causes More Leakage: Behavior Analysis

In [10]:
df.groupby("payment_failed")[["days_since_last_login", "Watch_Time_Hours"]].mean()

,days_since_last_login,Watch_Time_Hours
payment_failed,,
0,180.812402,521.359943
1,186.360370,408.993952


Users with failed payments exhibit higher inactivity and significantly lower watch time compared to users with successful payments. This reinforces the link between user disengagement and revenue leakage, highlighting inactivity as a key early warning signal.

## Defining Churn

In an OTT subscription platform, churn is typically associated with prolonged inactivity.
For this analysis, a user is considered churned if they have not logged in for more than 60 days.

This definition aligns with real-world subscription businesses where extended inactivity often precedes cancellation or non-renewal.

In [11]:
# Define churn based on inactivity threshold
df["churned"] = np.where(df["days_since_last_login"] > 60, 1, 0)

df["churned"].value_counts(normalize=True)

churned
1    0.8314
0    0.1686
Name: proportion, dtype: float64

Based on the inactivity threshold, approximately 83% of users are classified as churned, while about 17% are active. This indicates that a large portion of users in the dataset are inactive, which highlights significant retention and revenue risk for the platform.

## Revenue Loss by Churn Status

In [12]:
df.groupby("churned")[["revenue_expected", "revenue_loss"]].sum()

,revenue_expected,revenue_loss
churned,,
0,1885185,310098
1,9343315,1755604


Churned users account for the majority of revenue leakage, contributing approximately 1.76 million in lost revenue compared to about 0.31 million from active users. Despite representing inactive users, churned customers still account for a large portion of expected revenue, highlighting the high financial cost of churn.

## Churn Prediction Model
To predict churn, we start with a Logistic Regression model. This model is interpretable, fast to train, and well-suited for understanding which factors contribute most to churn risk.

In [14]:
features = [
    "Age",
    "Watch_Time_Hours",
    "days_since_last_login",
    "monthly_fee",
    "payment_failed"
]

X = df[features]
y = df["churned"]

X.head()

,Age,Watch_Time_Hours,days_since_last_login,monthly_fee,payment_failed
0,18,80.26,300,649,0
1,23,321.75,31,649,0
2,60,35.89,43,199,1
3,44,261.56,348,649,1
4,68,909.30,53,499,0


In [15]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

# Initialize model
model = LogisticRegression(max_iter=1000, class_weight="balanced")

# Train
model.fit(X_train, y_train)

# Predict
y_pred = model.predict(X_test)

# Evaluation
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       843
           1       1.00      1.00      1.00      4157

    accuracy                           1.00      5000
   macro avg       1.00      1.00      1.00      5000
weighted avg       1.00      1.00      1.00      5000



> Note: The perfect model performance is expected because the churn label is directly derived from user inactivity (`days_since_last_login`), which is also included as a feature.  
> This creates label leakage, making the prediction task trivial in this controlled simulation.

In [17]:
# Remove leakage-causing feature
features_no_leakage = [
    "Age",
    "Watch_Time_Hours",
    "monthly_fee",
    "payment_failed"
]

X_nl = df[features_no_leakage]
y = df["churned"]

X_nl.head()

,Age,Watch_Time_Hours,monthly_fee,payment_failed
0,18,80.26,649,0
1,23,321.75,649,0
2,60,35.89,199,1
3,44,261.56,649,1
4,68,909.30,499,0


In [18]:
from sklearn.model_selection import train_test_split

X_train_nl, X_test_nl, y_train, y_test = train_test_split(
    X_nl, y, test_size=0.2, random_state=42, stratify=y
)

In [19]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

model_nl = LogisticRegression(max_iter=1000, class_weight="balanced")
model_nl.fit(X_train_nl, y_train)

y_pred_nl = model_nl.predict(X_test_nl)

print(classification_report(y_test, y_pred_nl))

              precision    recall  f1-score   support

           0       0.17      0.69      0.28       843
           1       0.84      0.33      0.47      4157

    accuracy                           0.39      5000
   macro avg       0.51      0.51      0.37      5000
weighted avg       0.73      0.39      0.44      5000



In [20]:
confusion_matrix(y_test, y_pred_nl)

array([[ 585,  258],
       [2798, 1359]])

In [21]:
feature_importance_nl = pd.DataFrame({
    "feature": X_train_nl.columns,
    "coefficient": model_nl.coef_[0]
}).sort_values(by="coefficient", ascending=False)

feature_importance_nl

,feature,coefficient
3,payment_failed,0.212193
2,monthly_fee,0.000061
1,Watch_Time_Hours,0.000034
0,Age,-0.001559


After removing direct inactivity-based features, model performance drops significantly, which is expected. The model achieves a recall of 0.69 for active users but only 0.33 for churned users, indicating limited ability to detect churn using indirect behavioral signals alone.

The model correctly identifies a portion of active users but misclassifies many churned users as active. This highlights the difficulty of predicting churn without explicit recency signals and reflects real-world challenges in churn modeling.

Payment failure emerges as the strongest positive predictor of churn, reinforcing the link between billing issues and user attrition.
Watch time has a weaker but positive influence, while age shows a slight negative relationship with churn.
Subscription price has minimal direct impact on churn when isolated from engagement signals.

## Key Insight

While churn can be predicted perfectly using inactivity-based rules, predicting churn early—before users become inactive—is significantly more challenging.
Indirect signals such as payment failures and reduced engagement provide partial but imperfect early warnings.

## Business Recommendations
1. **Proactive Payment Recovery**
   Users with failed payments show a significantly higher churn risk. Introducing automated payment retries, reminders, and alternative payment options can directly reduce revenue leakage.

2. **Engagement-Based Retention Triggers**
   Declining watch time is an early signal of disengagement. Personalized content recommendations and re-engagement campaigns should be triggered before users become inactive.

3. **Prioritize High-Value Subscribers**
   Premium and Standard plan users contribute the highest absolute revenue loss. Retention efforts should be prioritized for these segments to maximize revenue recovery.

4. **Early-Warning Churn System**
   Even imperfect churn predictions can be valuable. Using probabilistic churn scores rather than hard labels allows product teams to intervene earlier and more effectively.

## Limitations

- The dataset is synthetic and may not capture all real-world user behaviors.
- Payment failures and churn labels were simulated based on assumptions rather than observed transactions.
- Lack of granular session-level or content-level data limits early churn prediction accuracy.